In [2]:
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# 800 total trials
# 400 mass questions; 400 force questions
# 337 actually active mass questions; 336 actually active mass questions 

In [3]:
pandas2ri.activate()

In [91]:
paths = ["../data/experiment1.rdata", "../data/experiment2.rdata"]
rdata_path = paths[0]

In [92]:
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [110]:
from constants import BASIC_TRAINING_COLS

all_trials = r['ls.clips']

if rdata_path == "../data/experiment1.rdata":
    responses = r['df.l'].query("yoker == 'NA'")[["trueMass", "trueRelationship"]]
    YOKED_PATH = "yoked_trials_exp1.h5"
    ALTERNATIVE_TRAINING_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y", "vx", "vy"]]
elif rdata_path == "../data/experiment2.rdata":
    responses = r['df.l'].query("experiment == 'experiment_4'")[["trueMass", "trueRelationship"]]
    YOKED_PATH = "yoked_trials_exp2.h5"
else:
    raise

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [97]:
responses.head()

,trueMass,trueRelationship
0,same,attract
1,A,none
2,A,none
3,same,attract
4,same,none


In [99]:
if os.path.exists(YOKED_PATH):
    os.remove(YOKED_PATH)

In [132]:
mass_trials = []
force_trials = []

trial_counter = 0

for participant in tqdm(all_trials):
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
            
        if rdata_path == "../data/experiment1.rdata":
            if set(ALTERNATIVE_TRAINING_COLS).issubset(trial.columns):
                trial = trial.rename({alt_col: col for col, alt_col in 
                                      zip(BASIC_TRAINING_COLS, ALTERNATIVE_TRAINING_COLS)}, axis="columns")
            else:
                print("Not all columns are available for participant")
                trial_counter += 1
                continue
     
        trial = trial[BASIC_TRAINING_COLS].copy()

        true_mass = responses.iloc[trial_counter]["trueMass"]
        trial["A"] = (true_mass == "A")
        trial["B"] = (true_mass == "B")
        trial["same"] = (true_mass == "same")

        true_relationship = responses.iloc[trial_counter]["trueRelationship"]
        trial["attract"] = (true_relationship == "attract")
        trial["none"] = (true_relationship == "none")
        trial["repel"] = (true_relationship == "repel")

        trial.to_hdf(path_or_buf=YOKED_PATH, key="trial_"+str(trial_counter))

        trial_counter += 1


  0%|          | 0/44 [00:00<?, ?it/s]/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
 75%|███████▌  | 33/44 [00:09<00:02,  4.43it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

 80%|███████▉  | 35/44 [00:09<00:01,  5.54it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

 89%|████████▊ | 39/44 [00:09<00:00,  8.00it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

 98%|█████████▊| 43/44 [00:10<00:00, 10.15it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

100%|██████████| 44/44 [00:10<00:00,  4.33it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant


# READING DATASET

In [133]:
from utils import read_dataset

In [134]:
yoked_dataset = read_dataset(YOKED_PATH)

In [135]:
yoked_dataset[0]

,o1.x,o1.y,o1.vx,o1.vy,o2.x,o2.y,o2.vx,o2.vy,o3.x,o3.y,...,o4.x,o4.y,o4.vx,o4.vy,A,B,same,attract,none,repel
0,1.192,2.228,8.057,0.709,0.825,0.784,-8.118,0.688,1.992,2.059,...,1.992,2.442,6.022,7.553,False,False,True,True,False,False
1,1.327,2.240,8.078,0.704,0.689,0.796,-8.108,0.694,2.092,1.910,...,2.093,2.567,6.003,7.476,False,False,True,True,False,False
2,1.462,2.252,8.095,0.700,0.554,0.807,-8.098,0.699,2.192,1.762,...,2.192,2.691,5.985,7.441,False,False,True,True,False,False
3,1.597,2.263,8.109,0.697,0.480,0.818,7.925,0.465,2.292,1.614,...,2.292,2.814,5.969,7.417,False,False,True,True,False,False
4,1.732,2.275,8.118,0.694,0.612,0.826,7.922,0.468,2.391,1.466,...,2.391,2.937,5.954,7.396,False,False,True,True,False,False
5,1.868,2.287,8.123,0.692,0.744,0.834,7.920,0.471,2.490,1.319,...,2.490,3.060,5.941,7.378,False,False,True,True,False,False
6,2.003,2.298,8.125,0.691,0.876,0.842,7.918,0.474,2.588,1.172,...,2.589,3.183,5.930,7.360,False,False,True,True,False,False
7,2.139,2.310,8.124,0.690,1.008,0.849,7.917,0.476,2.687,1.025,...,2.688,3.305,5.920,7.345,False,False,True,True,False,False
8,2.274,2.321,8.121,0.689,1.140,0.857,7.915,0.478,2.785,0.879,...,2.786,3.428,5.912,7.330,False,False,True,True,False,False
9,2.409,2.332,8.117,0.688,1.272,0.865,7.914,0.480,2.883,0.732,...,2.884,3.550,5.904,7.316,False,False,True,True,False,False


In [ ]:
read_dataset("passive_trials.h5")